<a href="https://colab.research.google.com/github/AndreNasci/ECO904/blob/main/ECO904_05_Supervisionado_Regressivo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Base de Dados Regressivo

[
Computer Hardware Data Set](http://archive.ics.uci.edu/ml/datasets/Computer+Hardware)

**Attribute Information:**

1. vendor name: 30
(adviser, amdahl,apollo, basf, bti, burroughs, c.r.d, cambex, cdc, dec,
dg, formation, four-phase, gould, honeywell, hp, ibm, ipl, magnuson,
microdata, nas, ncr, nixdorf, perkin-elmer, prime, siemens, sperry,
sratus, wang)
2. Model Name: many unique symbols
3. MYCT: machine cycle time in nanoseconds (integer)
4. MMIN: minimum main memory in kilobytes (integer)
5. MMAX: maximum main memory in kilobytes (integer)
6. CACH: cache memory in kilobytes (integer)
7. CHMIN: minimum channels in units (integer)
8. CHMAX: maximum channels in units (integer)
9. PRP: published relative performance (integer)
10. ERP: estimated relative performance from the original article (integer)

In [ ]:
import pandas as pd

url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/cpu-performance/machine.data'

hardware = pd.read_csv(url,names=['vendor','model','MYCT','MMIN','MMAX','CACH','CHMIN','CHMAX','PRP','ERP'])
hardware

## Analisando Dados

In [ ]:
# Colunas desnecessarias - não tem finalidade no treinamento
hardware = hardware.drop(['vendor','model'],axis=1)
hardware

In [ ]:
hardware.describe().T # taxas de variação completamente diferentes umas das outras
# trabalhar com escalas diferentes é ruim (escalas de valores) = necessita pré-processamento

In [ ]:
import seaborn as sns
g = sns.heatmap(abs(hardware.corr()),annot=True)
# correlação entre as colunas
# foi feito o absoluto da correlação, pois o que importa é o grau de correlação
# e não se ela é positiva ou negativa 

In [ ]:
# eliminar colunas com alto grau de correlação? # fazer isso após rodar todas as células 
# remover colunas com mais de 80% de correlação
# remover = [] # remover esse comentário dapós chegar ao fim do documento 
# inserir colunas e rodar de novo, como PRP
# hardware = hardware.drop(remover,axis=1)
# hardware

In [ ]:
# não possui distribuição gaussiana
g = hardware.hist()

In [ ]:
# foco especial no tutor
_ = hardware['ERP'].hist()

In [ ]:
# uso de autovetores
import seaborn as sns
from sklearn.decomposition import PCA

X = hardware[hardware.columns[:-1]].values
y = hardware[hardware.columns[-1]].values

Xp = PCA(n_components=2,random_state=42).fit_transform(X)

_ = sns.scatterplot(x=Xp[:,0],y=Xp[:,1],hue=y)
# o seaborn não é capaz de plotar tudo, apenas alguns steps para se 
# ter uma noção da concentração dos pontos

# Avaliação de Aprendizados Regressivos

In [ ]:
!pip install catboost # necessário ser instalado

In [ ]:
from sklearn import (dummy, ensemble, gaussian_process, kernel_ridge, 
                     linear_model, metrics, model_selection, neighbors, 
                     neural_network, svm, tree, preprocessing, pipeline)
from xgboost.sklearn import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from statistics import mean 
from tqdm.notebook import tqdm # informa o progresso 
import warnings # remove os warnings

preprocessador = [ # pré-processadores de ajuste de escala
    None,
    preprocessing.MinMaxScaler(),
    preprocessing.MaxAbsScaler(),
    preprocessing.Normalizer(),
    preprocessing.RobustScaler(),
    preprocessing.StandardScaler(),
]

aprendizados = [ # técnicas de aprendizado
    dummy.DummyRegressor(),
    linear_model.LinearRegression(),
    linear_model.Ridge(random_state=42),
    linear_model.SGDRegressor(random_state=42),
    linear_model.ElasticNet(random_state=42),
    ensemble.AdaBoostRegressor(random_state=42, max_depth=5), # arvore
    ensemble.BaggingRegressor(random_state=42, max_depth=5), #  arvore
    ensemble.ExtraTreesRegressor(random_state=42, max_depth=5), # arvore
    ensemble.GradientBoostingRegressor(random_state=42, max_depth=5), # arvore
    ensemble.RandomForestRegressor(random_state=42, max_depth=5),#  arvore
    ensemble.HistGradientBoostingRegressor(random_state=42, max_depth=5), # arvore
    gaussian_process.GaussianProcessRegressor(random_state=42),
    kernel_ridge.KernelRidge(),
    neighbors.KNeighborsRegressor(),
    neighbors.RadiusNeighborsRegressor(),
    neural_network.MLPRegressor(random_state=42),
    svm.LinearSVR(random_state=42), # regressor do support vector machine linear
    svm.NuSVR(),
    svm.SVR(),
    tree.DecisionTreeRegressor(random_state=42, max_depth=5), # arvore
    tree.ExtraTreeClassifier(random_state=42, max_depth=5), # arvore 
    XGBRegressor(random_state=42, max_depth=5), # arvore 
    LGBMRegressor(random_state=42, max_depth=5), # arvore
    CatBoostRegressor(verbose=0), # verbose != 0 -> imprime o funcionamento
]

# https://scikit-learn.org/stable/modules/model_evaluation.html#regression-metrics
# https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
metricas = ['max_error','neg_mean_absolute_error','neg_mean_squared_error','neg_mean_absolute_percentage_error','r2']
nomes = ['ME','MAE','MSE','MAPE','R2']
# ME = max error, maior erro encontrado na lista de erro (valor absoluto)
#   importante para identificar a presença de outliers
# MAE = erro médio absoluto = na mesma escala do tutor. Usado de base para
#   calcular outros erros (nesse caso, a escala é de 15-1280)
# MSE = erro médio quadrático: resolve o problema de sinais (todos os valores
#   serão positivos), e também destaca os outliers (pois está elevado a 2)
#   valores de outliers explodem
# MAPE = erro médio absoluto percentual: transforma-se a escala inicial (15-1280)
#   para uma escala de 0-100. Mas MAPE não é limitado a 100, ele pode tender a 
#   infinito. Isso se dá porque o valor é um float. Quanto menor melhor.
# R2 = quase uma acurácia. Quando MAPE == 0, R2 == 1. Quando MAPE == 1, R2 == 0.
#   Quando MAPE tende a inf, R2 tende a -inf. Quanto maior R2 melhor
# Contudo, MAPE e R2 não são completamente relacionados
# R2 é a mátrica padrão no sklearn do aprendizado supervisionado regressivo

sinal = [-1,-1,-1,-1,1]

pipes = []
# monta-se primeiro os pipelines e depois os treina
for p in preprocessador:
  for a in aprendizados:
    if (p != None):
      pipes.append(pipeline.make_pipeline(p,a))
    else:
      pipes.append(pipeline.make_pipeline(a))

warnings.filterwarnings('ignore') # desabilita os wanings
res = []
# treinamento de cada pipeline
for p in tqdm(pipes,desc='Pipelines'):

  cv_results = model_selection.cross_validate(p,X,y,cv=3,scoring=metricas)
  dn = {
      'nome':'.'.join([s[0] for s in p.steps]),
      'fitTime':mean(cv_results['fit_time']),
  }

  for m,n,s in zip(metricas,nomes,sinal):
    dn[n] = s * mean(cv_results['test_' + m])
  res.append(dn)

rDF = pd.DataFrame(res)

warnings.filterwarnings('default') # restaura warnings pro padrão

In [ ]:
# 10 melhores, de acordo com MAPE (% de erro)
rDF.sort_values('MAPE',ascending=True).head()

In [ ]:
# 10 melhores, de acordo com R2 (% de acerto)
rDF.sort_values('R2',ascending=False).tail()

## Ajustando Técnica

In [ ]:
# seleciona na coluna nome, todas as linhas que tiverem m1pregressor no nome
rDF[rDF['nome'].str.contains('mlpregressor')].sort_values('R2',ascending=False)

In [ ]:
from scipy.stats import randint, uniform as randfloat
# parâmetros de ajuste do modelo:
'''
hidden_layer_sizes=(100,), activation='relu', *, solver='adam', alpha=0.0001, batch_size='auto', 
learning_rate='constant', learning_rate_init=0.001, power_t=0.5, max_iter=200, shuffle=True, random_state=None, 
tol=0.0001, verbose=False, warm_start=False, momentum=0.9, nesterovs_momentum=True, early_stopping=False, 
validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08, n_iter_no_change=10, max_fun=15000
'''
clf = neural_network.MLPRegressor(random_state=42)

ajustes = {
    'hidden_layer_sizes': [(5,5,),(10,10,),(20,20,),(40,40,)], # regressivos = 1 único reurônio de saída
    #'alpha': randfloat(), # só funciona quando usamos fx sigmoidal
    'learning_rate_init': randfloat(),
    'max_iter':randint(200,1000), # 200 a 1000 iterações com o solver ADAM (vai sortear um valor entre 200 e 1000)
    # usa o MAPE negativo para identificar o melhor modelo
    # número total de iterações por vez rodada será max_iter x n_iter

}

rs = model_selection.RandomizedSearchCV(clf, ajustes, cv=3, # número de iterações para poder atingir o ponto ótimo
                                        scoring=['neg_mean_absolute_percentage_error','r2'],
                                        refit='neg_mean_absolute_percentage_error', 
                                        n_iter=200, random_state=42, verbose=0) #n~ de iterações do cross validation

#warnings.filterwarnings('ignore')
rs.fit(X,y)
#warnings.filterwarnings('default')

rs.best_score_, rs.best_params_

In [ ]:
dados = {
    'fit_time':rs.cv_results_['mean_fit_time'],
    'parametros':rs.cv_results_['params'],
    'MAPE':-rs.cv_results_['mean_test_neg_mean_absolute_percentage_error'],
    'R2':rs.cv_results_['mean_test_r2'],
    }
pd.DataFrame(dados).sort_values('R2',ascending=False).head()
# comparar valores de R2 e MAPE

In [ ]:
# melhor estimador baseado no valor de MAPE (ajustes do estimador)
rs.best_estimator_

## Avaliando Modelos

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import PredictionErrorDisplay

clf = rs.best_estimator_
print(str(clf))
y_pred = clf.predict(X)

fig, axs = plt.subplots(ncols=2, figsize=(8, 4))
PredictionErrorDisplay.from_predictions(
    y,
    y_pred=y_pred,
    kind="actual_vs_predicted",
    subsample=100,
    ax=axs[0],
    random_state=0,
)
axs[0].set_title("Actual vs. Predicted values")
PredictionErrorDisplay.from_predictions(
    y,
    y_pred=y_pred,
    kind="residual_vs_predicted",
    subsample=100,
    ax=axs[1],
    random_state=0,
)
axs[1].set_title("Residuals vs. Predicted Values")
_ = fig.suptitle("Plotting cross-validated predictions")
# visualização gráfica do erro

In [ ]:
rDF.sort_values('MAPE',ascending=True).head()

In [ ]:
from sklearn.model_selection import cross_val_predict

id_best = rDF.sort_values('MAPE',ascending=True).index[0]
clf= pipes[id_best]
print(str(clf))
y_pred =  cross_val_predict(clf, X, y, cv=10)

fig, axs = plt.subplots(ncols=2, figsize=(8, 4))
PredictionErrorDisplay.from_predictions(
    y,
    y_pred=y_pred,
    kind="actual_vs_predicted",
    subsample=100,
    ax=axs[0],
    random_state=0,
)
axs[0].set_title("Actual vs. Predicted values")
PredictionErrorDisplay.from_predictions(
    y,
    y_pred=y_pred,
    kind="residual_vs_predicted",
    subsample=100,
    ax=axs[1],
    random_state=0,
)
axs[1].set_title("Residuals vs. Predicted Values")
_ = fig.suptitle("Plotting cross-validated predictions")
# observar como o modelo concentra mais os pontos sobre a reta em comparação
# com o anterior 

### Avaliando Árvores de Decisão

In [ ]:
# árvores tem o maior rendimento entre as técnicas clássicas
# perde pra deep learning, embora seu consumo energético seja significativamente menor
from sklearn.tree import export_graphviz # biblioteca vetorial do python (cria uma representação gráfica)
from pydotplus import graph_from_dot_data
from IPython.display import Image

def ImprimeArvore(tree):
  dot_data = export_graphviz(tree, feature_names = hardware.columns[:-1],
      out_file=None, filled=True, rounded=True,
      special_characters=True,
      proportion=False, impurity=False,
  )

  graph = graph_from_dot_data(dot_data)
  png = graph.create_png()
  display(Image(png))

In [ ]:
# mostra os melhores modelos
rDF[rDF['nome'].str.contains('randomforestregressor')].sort_values('R2',ascending=False)

In [ ]:
# 43 minmaxscaler.decisiontreeregressor
pp = pipes[43] # buscando o melhor modelo

arvore = pp.steps[1][1] # primeiro elemento é o nome da técnica, segundo é o modelo

# treinando novamente o modelo, porque quando é feito no gridSearch ele apenas
# treina o modelo, mas não o retorna
arvore.fit(X,y)
imprimeArvore(arvore)
# analisar depth da árvore 

In [ ]:
# repetir esse procedimento para outras árvores
tree = pipes[96].steps[1][1]
tree.fit(X,y)

ImprimeArvore(tree.estimators_[30])

In [ ]:
pp = pipes[105] # floresta
floresta = pp.steps[1][1]
floresta.fit(X,y)
len(floresta.estimators_) # quantidade de arvores na floresta
# cada árvore se especializa em uma parte da tabela, o que, em geral,
# dá um resultado melhor do que árvores sozinhas 

In [ ]:
# conferindo uma das árvores
arvore = floresta.estimators_[39]
ImprimeArvore(arvore)
# note que o random_state é o mesmo para as 100 arvores porque foi setado 42 anteriormente
arvore = floresta.estimators_[58]
ImprimeArvore(arvore)
# cada uma das árvores é treinada com um pacote de dados diferentes
# a fim de gerar a especialização

In [ ]:
rDF[rDF['nome'].str.contains('decisiontreeregressor')].sort_values('R2',ascending=False)

# Atividade

Refaça os passos anteriores com o dataset abaixo:

In [ ]:
import pandas as pd

# volume de tráfico de pessoas do metrô interestatudal 
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/00492/Metro_Interstate_Traffic_Volume.csv.gz'

metro = pd.read_csv(url, parse_dates=['date_time']) # colab é capaz de ler um arquivo .gz diretamente
# e fazer o typecast de date_time automaticamente na leitura
metro.head()

In [ ]:
# 48204 linhas = 48204 horas registradas
# 9 colunas
# date_time nos dá diversos detalhes em apenas uma coluna 
# apesar de estar em object, podemos converte-lo na propria leitura
metro.shape

In [ ]:
print(metro['holiday'].unique())
print(metro['weather_main'].unique())
print(metro['weather_description'].unique())

In [ ]:
mt = metro.copy()
categorias = {}
for col in mt.columns:
  if mt[col] 
    categorias[col] = tipos
print(categorias)

In [ ]:
mt

In [ ]:
mt['hora'] = mt['date_time'].dt.hour() # dt converte de numpy para um tipo python
mt['diaSemana'] = mt['date_time'].dt.weekday()
mt['semanaAno'] = mt['date_time'].dt.isocalendar().week  
mt = mt.drop('date_time', axis=1)

In [ ]:
mt

In [ ]:
X = mt.drop('traffic_volume', axis=1).values
# y = 